### `gold_kpis_gerais` – KPIs Operacionais da Frota  
**1 linha | ~3 segundos | Atualiza a cada 10 min**

| KPI | Fonte | Descrição |
|-----|-------|-----------|
| `frota_ativa` | `gold_posicao_atual` | Nº de ônibus com posição recente |
| `onibus_acessiveis` | `gold_posicao_atual` | Nº de veículos com acessibilidade |
| `perc_acessivel` | Cálculo | `% acessíveis = acessíveis / frota_ativa` |
| `total_previsoes` | `gold_previsao_paradas` | Nº de ETAs ativas no momento |
| `atualizado_em` | `current_timestamp()` | Horário da última atualização |

**Tabela final:** `workspace.sptrans.gold_kpis`  
**Uso:** Cards no dashboard, relatórios gerenciais, alertas  
**Leve e rápido:** só agregações — **sem join pesado, sem Haversine**  
**Atualiza em 3s** após `gold_previsao_paradas`



In [0]:
# ==============================================================
# GOLD_KPIS_GERAIS – 1 LINHA, 3 SEGUNDOS
# ==============================================================

from pyspark.sql.functions import countDistinct, sum, when, col, lit, current_timestamp, round

pos = spark.table("workspace.sptrans.gold_posicao_atual")
prev = spark.table("workspace.sptrans.gold_previsao_paradas")

kpis = pos.agg(
    countDistinct("prefixo").alias("frota_ativa"),
    sum(when(col("acessivel"), 1).otherwise(0)).alias("onibus_acessiveis")
).withColumn("perc_acessivel", round(col("onibus_acessiveis") * 100.0 / col("frota_ativa"), 1)) \
 .withColumn("total_previsoes", lit(prev.count())) \
 .withColumn("atualizado_em", current_timestamp())

kpis.write.format("delta").mode("overwrite").saveAsTable("workspace.sptrans.gold_kpis")

print("KPIs atualizados! 1 linha salva.")
display(kpis)

KPIs atualizados! 1 linha salva.


frota_ativa,onibus_acessiveis,perc_acessivel,total_previsoes,atualizado_em
11335,11332,100.0,5766,2025-11-11T21:31:11.581Z
